# Setup


In [ ]:
!nvidia-smi

Create a `HOME` constant


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

Install YOLOv11 via Ultralytics

In [ ]:
%pip install "ultralytics<=8.3.40" supervision roboflow
!pip install roboflow
import ultralytics
ultralytics.checks()

Create folder for dataaset and import the dataset from Roboflow

# Training

## Import dataset from Roboflow

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

from google.colab import userdata
from roboflow import Roboflow
rf = Roboflow(api_key="API KEY")
project = rf.workspace("WORKSPACE ID").project("PROJECT ID")
version = project.version("DATASET VERSION")
dataset = version.download("yolov11")


## Import dataset from Google Drive

In [ ]:
%cd {HOME}

# Replace 'path/to/your/dataset.zip' with the actual path to your zip file in Google Drive
zip_path = '/content/drive/MyDrive/Sem 8/FYP 2/Bovine Disease Detection-Combined.v1i.yolov11.image_balancedv2_1300.zip'
dataset_dir = os.path.join(HOME, 'datasets', 'Bovine Disease Detection-Combined.v1i.yolov11.image_balancedv2_1300')

# Unzip the dataset
!mkdir -p "{dataset_dir}"
!unzip -q "{zip_path}" -d "{dataset_dir}"

dataset = dataset_dir

print(f"Dataset unzipped to: {dataset}")

## Training

In [ ]:
%cd {HOME}

# Replace "{dataset.location}/data.yaml" to "{dataset}/data.yaml" for datsets imported from Google Drive
!yolo task=detect mode=train model=yolo11s.pt data="{dataset.location}/data.yaml" epochs=100 patience=10 cos_lr=True imgsz=640 plots=True optimizer="AdamW"

## Examine result outputs

In [ ]:
!ls {HOME}/runs/detect/train/

## Confusion Matrix


In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/confusion_matrix.png', width=600)

## Training metrics

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/results.png', width=600)

In [ ]:
from IPython.display import Image as IPyImage

IPyImage(filename=f'{HOME}/runs/detect/train/val_batch0_pred.jpg', width=600)

## Validate model


In [ ]:
!yolo task=detect mode=val model={HOME}/runs/detect/train/weights/best.pt data="{dataset.location}/data.yaml"

# Inference

In [ ]:
!yolo task=detect mode=predict model={HOME}/runs/detect/train/weights/best.pt conf=0.25 source="{dataset.location}/test/images" save=True

In [ ]:
import glob
import os
from IPython.display import Image as IPyImage, display

latest_folder = max(glob.glob(f'{HOME}/runs/detect/predict*/'), key=os.path.getmtime)
for img in glob.glob(f'{latest_folder}/*.jpg')[:3]:
    display(IPyImage(filename=img, width=600))
    print("\n")

In [ ]:
from google.colab import files
import os
import shutil

# Define the source folder path
source_folder = os.path.join(HOME, 'runs')

# Define the destination zip file path
destination_zip = 'runs.zip'

# Create the zip file
shutil.make_archive(destination_zip.replace('.zip', ''), 'zip', source_folder)

# Download the zip file
files.download(destination_zip)

# Export model to Roboflow

In [ ]:
project.version(dataset.version).deploy(model_type="yolov11", model_path=f"{HOME}/runs/detect/train/")

In [ ]:
!pip install inference

In [ ]:
import os, random, cv2
import supervision as sv
import IPython
import inference

model_id = project.id.split("/")[1] + "/" + dataset.version
model = inference.get_model(model_id, userdata.get('ROBOFLOW_API_KEY'))

# Location of test set images
test_set_loc = dataset.location + "/test/images/"
test_images = os.listdir(test_set_loc)

# Run inference on 4 random test images, or fewer if fewer images are available
for img_name in random.sample(test_images, min(4, len(test_images))):
    print("Running inference on " + img_name)

    # Load image
    image = cv2.imread(os.path.join(test_set_loc, img_name))

    # Perform inference
    results = model.infer(image, confidence=0.4, overlap=30)[0]
    detections = sv.Detections.from_inference(results)

    # Annotate boxes and labels
    box_annotator = sv.BoxAnnotator()
    label_annotator = sv.LabelAnnotator()
    annotated_image = box_annotator.annotate(scene=image, detections=detections)
    annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections)

    # Display annotated image
    _, ret = cv2.imencode('.jpg', annotated_image)
    i = IPython.display.Image(data=ret)
    IPython.display.display(i)